# 第3部 第9章 ロジスティック回帰モデル

## はじめに

## フォルダ構造とユーティリティ関数、ライブラリimport
リンク集の記事にフォルダ構造とユーティリティ関数、ライブラリimportを掲載しました。\
準備としてそちらのページをご覧ください。
1. [フォルダ構造とユーティリティ関数]()
1. [ライブラリimport]()

## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
cmap = plt.get_cmap("tab10")
%matplotlib inline
import seaborn as sns
import arviz as az

# plotの設定
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}

file_path = "../mod/rcParams.json"
with open(file_path) as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 9.1 本章の目的と概要
参考書籍から引用します。\
RとStanやbrmsの部分をPythonとNumPyroに読み替えてください。

> **テーマ**\
> 本章ではロジスティック回帰モデルの推定を行います.

> **目的**\
> 一般化線形モデルの枠組みで「あり・なし」といった二値データを扱う標準的な手法を紹介することを目的として,本性を執筆しました.

> **概要**\
> モデルの構造 → 分析の準備 → データの読み込みと可視化\
> → brms によるロジスティック回帰モデルの推定 → 推定されたモデルの解釈\
> → (補足)Stanファイルの実装 → (補足)試行回数が常に1の場合の実装

## 9.2 モデルの実装
植木鉢に植えた10粒の種子のうち発芽した種子の数 $y_{i}$ と日照状態と栄養素の量の組 $( x_{1}, x_{2} )$ との関係をモデル化します。\
書籍のモデル構造式を掲載します。\
ここで日照状態 $x_{1}$ は植木鉢に日が当たっていれば $1$ ,そうでない場合に $0$ を取る二値変数です。\
栄養素の量 $x_{2}$ は連続値変数です。

$$
\begin{aligned}
logit \left ( p_{i} \right ) =& \beta_{0} + \beta_{1} x_{i, 1} + \beta_{2} x_{i, 2}\\
y_{i} \sim& Binom \left ( 10, p_{i}  \right )
\end{aligned}
$$

この構造式に至るための仮定を補足します。\
発芽した種子の数 $y_{i}$ は $0$ 以上の整数なので $y_{i}$ は離散的な確率分布に従います。\
さらに植えたのは10粒なので $y_{i}$ の最大値は10です。\
これらの条件を満たす確率分布として二項分布を選びます。\
二項分布をは総数 $n$ と確率 $p_{i}$ をパラメータに持つ離散的な確率分布です。

$$
\begin{aligned}
y_{i} \sim& Binom \left ( n = 10, p_{i}  \right )\\
\lambda_{i} >& 0
\end{aligned}
$$

日照状態と栄養素の量の組の影響をモデル化したいので1次関数 $\mu_{i} = \beta_{0} + \beta_{1} x_{i, 1} + \beta_{2} x_{i, 2}$ を確率分布に組込みたいです。\
しかし1次関数は負の数を返す可能性がありそのままでは確率 $p_{i}$ に代入できません。\
ロジスティック関数に1次関数を渡した値を二項分布をのパラメータにすることで、この悩みを解決します。\
NumPyroでモデルを作成する場合は $ logit ( p_{i} ) = \mu_{i}$ の式を使った方が安定します。

$$
\begin{aligned}
y_{i} \sim& Binom \left ( n = 10, p_{i}  \right )\\
p_{i} \equiv& logistic ( \mu_{i} )\\
=& \frac{1}{ 1 + \exp (- \mu_{i})}\\
logit ( p_{i} ) =& \mu_{i}\\
\mu_{i} =& \beta_{0} + \beta_{1} x_{i, 1} + \beta_{2} x_{i, 2}
\end{aligned}
$$

植木鉢に植えた10粒の種子のうち発芽した種子の数 $y_{i}$ と日照状態と栄養素の量の組 $( x_{1}, x_{2} )$ との関係をモデル化できました。

## 9.3 分析の準備
ライブラリの準備をしている節です。\
省略します。

## 9.4 データの読み込みと可視化
データを読み込んで概要を把握します。

In [ ]:
# データを読み込む
df = pl.read_csv("../data/3-9-1-germination.csv")
display(df.head())

# サンプルサイズを表示する
print( len(df) )

発芽した種子の数と栄養素の量の関係の散布図を日照状態で層別してプロットします。\
栄養素の量が多いほど発芽した種子の数が増えています。\
さらに植木鉢に日が当たっていると発芽しやすいこともわかりました。

In [ ]:
ax = sns.scatterplot(data = df, x = "nutrition", y = "germination", hue = "solar")
ax.set_title("図3.9.1 種子の発芽数と日照状態・栄養素の量の散布図")

モデルの構造の通りのダミー変数を作成します.

In [ ]:
df = (
    df
    .with_columns([
        pl.when( pl.col("solar") == "sunshine" ).then(1)
        .otherwise(0)
        .alias("x1"),
    ])
)
display(df.head())

numpyroに渡すためにjax.numpy配列に変換します。

In [ ]:
X1 = jnp.array(df["x1"], dtype = int)
X2 = jnp.array(df["nutrition"], dtype = float)
Y = jnp.array(df["germination"], dtype = int)

## 9.5 brmsによるロジスティック回帰モデルの推定
NumPyroでモデルを作成します。

In [ ]:
def model_logistic(X1, X2, Y = None):
    '''
        第3部 9章 の釣獲尾数
    '''
    # 説明変数をモデルに明示する
    # ベクトル化(学習用データを確率変数に割り当てるためのNumPyroのお作法)
    N = len(X1)
    with numpyro.plate("N", N):
        X1 = numpyro.deterministic("X1", X1)
        X2 = numpyro.deterministic("X2", X2)
    # 確率変数のパラメータの事前分布を設定する
    β0 = numpyro.sample("β0", dist.Normal(loc = 0, scale = 3))
    β1 = numpyro.sample("β1", dist.Normal(loc = 0, scale = 3))
    β2 = numpyro.sample("β2", dist.Normal(loc = 0, scale = 3))
    # リンク関数を定義する
    with numpyro.plate("N", N):
        μ = numpyro.deterministic("μ", β0 + β1 * X1 + β2 * X2)
        p = numpyro.deterministic("p", 1 / (1 + jnp.exp(-μ)) )
    # 目的変数 y は説明変数 x の値に応じた値 λ をパラメータとするポアソン分布に従うと仮定します。
    with numpyro.plate("N", N):
        numpyro.sample("Y", dist.Binomial(total_count = 10, logits = μ), obs = Y)

モデルをプロットします。

In [ ]:
model_args = {
    "X1": X1,
    "X2": X2,
    "Y": Y,
}
try_render_model(model_logistic, render_name = "発芽した種子の数と日照状態・栄養素の量の関係", **model_args)

ユーティリティ関数にモデルを渡してサンプリングします。

In [ ]:
model_args = {
    "X1": X1,
    "X2": X2,
    "Y": Y
}
mcmc = run_mcmc(
    model_logistic,
    num_chains = 4,
    num_warmup = 1000,
    num_samples = 1000,
    thinning = 1,
    seed = 42,
    target_accept_prob = 0.9,
    **model_args
)
idata = az.from_numpyro(mcmc, log_likelihood = False)

サンプリングが上手くいったか確認します。

In [ ]:
az.plot_trace(idata, compact = False, var_names = ["β0", "β1", "β2"])
plt.tight_layout()

MCMCサンプルの概要を確認します。\
HDIを見ると $\beta 1$ が正の値なので植木鉢に日が当たっていると発芽率が高まり、 $\beta 2$ が正の数なので栄養素の量が多いほど発芽率が高まるようです。

In [ ]:
az.summary(idata, var_names = ["β0", "β1", "β2"])

## 8.6 推定されたモデルの解釈
ロジスティック回帰モデルの解釈では確率そのものよりも確率のオッズ $p/(1-p)$ を利用することが多いです。\
参考書籍に倣って計算します。

In [ ]:
logit_01 = -7.736 + 3.902 * 0 + 0.693 * 2
p_01 = 1 / (1 + np.exp(-logit_01) )
odds_01 = p_01 / (1 - p_01)
print(
    "1. 植木鉢に日が当たっていない, 栄養素の量 2 -> logit(p):", np.round( logit_01, decimals = 2 ),
    "-> 確率p:", np.round( p_01, decimals = 4 ),
    "-> オッズ:", np.round( odds_01, decimals = 4 ),
)
logit_02 = -7.736 + 3.902 * 1 + 0.693 * 2
p_02 = 1 / (1 + np.exp(-logit_02) )
odds_02 = p_02 / (1 - p_02)
print(
    "2. 植木鉢に日が当たっている, 栄養素の量 2 -> logit(p):", np.round( logit_02, decimals = 2 ),
    "-> 確率p:", np.round( p_02, decimals = 4 ),
   "-> オッズ:", np.round( odds_02, decimals = 4 ),
)
logit_03 = -7.736 + 3.902 * 1 + 0.693 * 3
p_03 = 1 / (1 + np.exp(-logit_03) )
odds_03 = p_03 / (1 - p_03)
print(
    "3. 植木鉢に日が当たっている, 栄養素の量 3 -> logit(p):", np.round( logit_03, decimals = 2 ),
    "-> 確率p:", np.round( p_03, decimals = 4 ),
    "-> オッズ:", np.round( odds_03, decimals = 4 ),
)

print("1. と 2. のオッズ比", np.round( odds_02 / odds_01, decimals = 2 ) )
print("2. と 3. のオッズ比", np.round( odds_03 / odds_02, decimals = 2 ) )

## 8.7 回帰曲線の図示
天気別の事後予測分布を計算します。\
NumPyroは少し準備が必要です。

In [ ]:
# 予測用のデータ
list_temperature = list( range( int(df["nutrition"].min()), int(df["nutrition"].max()+1) ) )
X2 = jnp.array(list_temperature * 2, dtype = float)
N_X = len(list_temperature)
model_args = {
    "X1": jnp.array([0] * N_X + [1] * N_X, dtype = int),
    "X2": X2,
    "Y": None,
}
# 予測データの目的変数名
var_name = "Y"

# 事後予測分布を計算する
idata = compute_posterior_predictive_distribution(
    model_logistic,
    mcmc,
    var_name = "Y",
    seed = 42,
    hdi_prob = 0.95,
    log_likelihood = False,
    **model_args,
)

# 事後予測平均
y_ppc_mean = np.array( idata["posterior_predictive"][var_name].mean(dim = ("chain", "draw")) )
# 事後予測のHDI
hdi_ppc = az.hdi(
    idata,                          # InferenceData 全体を渡す
    group = "posterior_predictive", # 計算対象グループを指定
    var_names = [var_name],         # 対象変数
    hdi_prob = 0.95,            # 区間幅
)
y_ppc_low = np.array( hdi_ppc[var_name][:, 0] )
y_ppc_high = np.array( hdi_ppc[var_name][:, 1] )

# データを保存する
df_pred = pl.DataFrame({
    "solar": ["shade"] * N_X + ["sunshine"] * N_X,
    "nutrition": np.array(X2),
    "y_pred": y_ppc_mean,
    "y_ppc_low": y_ppc_low,
    "y_ppc_high": y_ppc_high,
})

散布図と事後予測分布をプロットします。\
晴れの日の売り上げが好調であることがわかります。

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1)
for idx, solar in enumerate(df["solar"].unique().sort()):
    expr = pl.col("solar") == solar
    sns.scatterplot(data = df.filter(expr), x = "nutrition", y = "germination", ax = ax, color = cmap(idx), label = solar)
    sns.lineplot(data = df_pred.filter(expr), x = "nutrition", y = "y_pred", ax = ax, color = cmap(idx))
    ax.fill_between(
        x = df_pred.filter(expr)["nutrition"], y1 = df_pred.filter(expr)["y_ppc_high"], y2 = df_pred.filter(expr)["y_ppc_low"],
        color = cmap(idx), alpha=0.2
    )
plt.suptitle("図3.9.2 ロジスティック回帰モデルの回帰曲線: 信用区間付き")

## 9.8 補足: ロジスティック回帰モデルのためのStanファイルの実装
省略します。

## 9.9 補足: 試行回数が常に1の場合
省略します。

## 終わりに